In [27]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


# New section

In [28]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# to list files
import glob

# for numerical analysis
import numpy as np 
# to store and process in a dataframe
import pandas as pd 

# for ploting graphs
import matplotlib.pyplot as plt
# advancec ploting
import seaborn as sns

# image processing
import matplotlib.image as mpimg

# train test split
from sklearn.model_selection import train_test_split
# model performance metrics
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.layers import Activation, ZeroPadding3D, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
import tensorflow as tf
# utility functions
from tensorflow.keras.utils import to_categorical, plot_model
# process image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
# sequential model
from tensorflow.keras.models import Sequential
# layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# callback functions
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.applications import InceptionV3
import tensorflow.keras.utils as utils
#from utils import img_to_array
from glob import glob
from tqdm.notebook import tqdm
import pathlib
import csv
import os

from sklearn.preprocessing import LabelEncoder
from pathlib import Path

In [30]:
!unzip gdrive/My\ Drive/data/pert_data2.zip 

Archive:  gdrive/My Drive/data/pert_data2.zip
replace pert_data/0_0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: pert_data/0_0.png       
  inflating: pert_data/1_1.png       
  inflating: pert_data/10_0.png      
  inflating: pert_data/100_0.png     
  inflating: pert_data/1000_0.png    
  inflating: pert_data/1001_1.png    
  inflating: pert_data/1002_1.png    
  inflating: pert_data/1003_0.png    
  inflating: pert_data/1004_0.png    
  inflating: pert_data/1005_0.png    
  inflating: pert_data/1006_0.png    
  inflating: pert_data/1007_0.png    
  inflating: pert_data/1008_1.png    
  inflating: pert_data/1009_1.png    
  inflating: pert_data/101_0.png     
  inflating: pert_data/1010_0.png    
  inflating: pert_data/1011_0.png    
  inflating: pert_data/1012_1.png    
  inflating: pert_data/1013_1.png    
  inflating: pert_data/1014_1.png    
  inflating: pert_data/1015_1.png    
  inflating: pert_data/1016_1.png    
  inflating: pert_data/1017_1.png    
  inflating: 

In [31]:
BATCH_SIZE = 100  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels
TARGET_SIZE = 64
EPOCHS = 10

In [32]:
directory = '/content/pert_data/'
ds_train = tf.data.Dataset.list_files(str(pathlib.Path(directory + "*.png")))
def process_path(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=3)
    label = tf.strings.split(file_path, "/")[-1]
    label = tf.strings.split(label, ".")[0]
    label = tf.strings.split(label, "_")[1]
    label = tf.strings.to_number(label, out_type=tf.int64)
    return image, label


ds_train = ds_train.map(process_path).batch(BATCH_SIZE)
model = keras.models.load_model('gdrive/MyDrive/data/best_model_R50.hdf5')
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 


In [34]:
# Instantiate an early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', 
                               min_delta = 0.01,
                               patience=2)

# Instantiate a model checkpoint callback
model_save = ModelCheckpoint('Robust_model_R50.hdf5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True)
checkpoint_path = 'gdrive/MyDrive/data/checkpoint/'

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(ds_train, epochs=3, verbose=1,callbacks=[cp_callback])

Epoch 1/3
35/35 [==============================] - ETA: 0s - loss: 0.0233 - accuracy: 0.9926
Epoch 1: saving model to gdrive/MyDrive/data/checkpoint/
35/35 [==============================] - 17s 495ms/step - loss: 0.0233 - accuracy: 0.9926
Epoch 2/3
35/35 [==============================] - ETA: 0s - loss: 0.0213 - accuracy: 0.9931
Epoch 2: saving model to gdrive/MyDrive/data/checkpoint/
35/35 [==============================] - 18s 505ms/step - loss: 0.0213 - accuracy: 0.9931
Epoch 3/3
35/35 [==============================] - ETA: 0s - loss: 0.0054 - accuracy: 0.9986
Epoch 3: saving model to gdrive/MyDrive/data/checkpoint/
35/35 [==============================] - 18s 518ms/step - loss: 0.0054 - accuracy: 0.9986


In [35]:
model.save('gdrive/MyDrive/data/robust_checkpoint2.h5')